In [ ]:
import os
import cv2
import numpy as np
import random
from tqdm.notebook import tqdm
import shutil

In [ ]:
def augment_frame(
    frame,
    frame_width,
    frame_height,
    translation_percent,
    shear_factor,
    noise_stddev,
    scale_factor,
):
    """
    Apply translation, shearing, scaling, and Gaussian noise to a frame.
    """
    # Translation
    translation_pixels = int(frame_width * translation_percent)
    translation_matrix = np.float32([[1, 0, translation_pixels], [0, 1, 0]])
    translated_frame = cv2.warpAffine(
        frame, translation_matrix, (frame_width, frame_height)
    )

    # Shearing
    shear_matrix = np.float32([[1, shear_factor, 0], [0, 1, 0]])
    sheared_frame = cv2.warpAffine(
        translated_frame, shear_matrix, (frame_width, frame_height)
    )

    # Scaling
    scaled_width = int(frame_width * scale_factor)
    scaled_height = int(frame_height * scale_factor)
    scaled_frame = cv2.resize(sheared_frame, (scaled_width, scaled_height))

    # Crop or pad to maintain original size
    if scale_factor > 1.0:
        start_x = (scaled_width - frame_width) // 2
        start_y = (scaled_height - frame_height) // 2
        scaled_frame = scaled_frame[
            start_y : start_y + frame_height, start_x : start_x + frame_width
        ]
    else:
        pad_x = (frame_width - scaled_width) // 2
        pad_y = (frame_height - scaled_height) // 2
        scaled_frame = cv2.copyMakeBorder(
            scaled_frame,
            pad_y,
            frame_height - scaled_height - pad_y,
            pad_x,
            frame_width - scaled_width - pad_x,
            cv2.BORDER_CONSTANT,
            value=[0, 0, 0],
        )

    # Add Gaussian noise
    noise = np.random.normal(0, noise_stddev, frame.shape).astype(np.float32)
    noisy_frame = np.clip(scaled_frame + noise, 0, 255).astype(np.uint8)

    return noisy_frame

In [ ]:
def process_video(video_path, output_dir, num_copies=3, **augmentation_params):
    """
    Process a single video and save augmented copies.
    """

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video {video_path}")
        return

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")

    # Save augmented copies
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    for i in range(num_copies):
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reset to the first frame
        augmented_output_path = os.path.join(
            output_dir, f"{video_name}_augmented_{i + 1}.mp4"
        )
        out_augmented = cv2.VideoWriter(
            augmented_output_path, fourcc, fps, (frame_width, frame_height)
        )

        # Generate random parameters for this augmented copy
        translation_percent = random.uniform(*augmentation_params["translation_range"])
        shear_factor = random.uniform(*augmentation_params["shear_range"])
        noise_stddev = random.uniform(*augmentation_params["noise_stddev_range"])
        scale_factor = random.uniform(*augmentation_params["scale_range"])

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            augmented_frame = augment_frame(
                frame,
                frame_width,
                frame_height,
                translation_percent,
                shear_factor,
                noise_stddev,
                scale_factor,
            )
            out_augmented.write(augmented_frame)

        out_augmented.release()

    cap.release()

In [ ]:
def augment_training_videos(
    train_folder, output_train_folder, num_copies=3, **augmentation_params
):
    """
    Augment all videos in the training folder and save them in the output directory.
    """
    for label in tqdm(
        os.listdir(train_folder), desc="Augmenting Training Folders", unit="folder"
    ):
        label_input_folder = os.path.join(train_folder, label)
        label_output_folder = os.path.join(output_train_folder, label)

        if not os.path.isdir(label_input_folder):
            continue

        os.makedirs(label_output_folder, exist_ok=True)
        for video_file in tqdm(
            os.listdir(label_input_folder), desc=f"Processing {label}", leave=False
        ):
            video_path = os.path.join(label_input_folder, video_file)
            process_video(
                video_path,
                label_output_folder,
                num_copies=num_copies,
                **augmentation_params,
            )
            shutil.copy(video_path, label_output_folder)


def copy_test_videos(test_folder, output_test_folder):
    """
    Copy all videos from the test folder to the output directory without augmentation.
    """
    if not os.path.exists(test_folder):
        print(f"Test folder does not exist: {test_folder}")
        return

    shutil.copytree(test_folder, output_test_folder, dirs_exist_ok=True)
    print(f"Copied test videos to {output_test_folder}")


In [ ]:
def main(input_root, output_root, num_copies=3, **augmentation_params):
    """
    Main function to augment training videos and copy test videos.
    """
    train_folder = os.path.join(input_root, "train")
    test_folder = os.path.join(input_root, "test")
    output_train_folder = os.path.join(output_root, "train")
    output_test_folder = os.path.join(output_root, "test")

    os.makedirs(output_train_folder, exist_ok=True)
    os.makedirs(output_test_folder, exist_ok=True)

    augment_training_videos(
        train_folder, output_train_folder, num_copies=num_copies, **augmentation_params
    )

    copy_test_videos(test_folder, output_test_folder)


In [ ]:
# Define input and output directories
input_root = "data_processed/"
output_root = "data_augmented/"
os.makedirs(output_root, exist_ok=True)

# Define augmentation parameters
augmentation_params = {
    "translation_range": (-0.2, 0.2),
    "shear_range": (-0.2, 0.2),
    "noise_stddev_range": (5, 25),
    "scale_range": (0.6, 1.2),
}

# Run the pipeline
main(input_root, output_root, num_copies=3, **augmentation_params)

Augmenting Training Folders:   0%|          | 0/30 [00:00<?, ?folder/s]

Processing adik:   0%|          | 0/8 [00:00<?, ?it/s]

Processing anak:   0%|          | 0/8 [00:00<?, ?it/s]

Processing besar:   0%|          | 0/8 [00:00<?, ?it/s]

Processing buka:   0%|          | 0/8 [00:00<?, ?it/s]

...

Processing sedikit:   0%|          | 0/8 [00:00<?, ?it/s]

Processing selamat:   0%|          | 0/8 [00:00<?, ?it/s]

Processing senyum:   0%|          | 0/8 [00:00<?, ?it/s]

Processing teman:   0%|          | 0/7 [00:00<?, ?it/s]

Processing tidur:   0%|          | 0/7 [00:00<?, ?it/s]